# MultipleRuns_fixed_p  
The purpose of this notebook is to execute multiple runs for different values of $\beta s$ as it is quite a time-consuming task (>1h on my laptop). Results are saved externally and then retrieved in the main notebook.

In [ ]:
import numpy as np
from numpy import random
import networkx as nx
import pandas as pd
import random
import collections
import matplotlib.pyplot as plt
from itertools import combinations
import os
import json

In [ ]:
#Model
class Model():
    def __init__(self, simplices):
        #Structure
        self.simplices = simplices
        self.nodes = list({node for simplex in simplices for node in simplex}) #it returns the list of unique nodes
        self.N = len(self.nodes)

        #Time
        self.t = 0
    
    def SetInitialConditions(self, beta, p, n_A, verbose=False):
        """
        This function sets the intial conditions.

        :verbose: if True, a string containing set-up details is printed.
        """
        #Params
        self.beta = beta
        self.p = p

        #Nodes' vocabularies (opinions)
        self.opinions = {}

        #Generating the committed minority 
        N_p = int(self.N*self.p) #number of committed individuals
        committed = random.sample(self.nodes, N_p) #random sample to create the committed agents
        for n in self.nodes:
            if n in committed:
                self.opinions[n]=frozenset(["A"]) #the committed individuals do not change their mind
        
        #Computing the size of intial communities
        N_A = int(self.N*n_A)
        N_B = self.N - N_A - N_p

        #Assigning opinions to individual agents 
        opinions_to_assign = ['A']*N_A + ['B']*N_B #create a list of opinions that follows the communities' sizes
        random.shuffle(opinions_to_assign) #shuffle the list
        noncommitted = set(self.nodes)-set(committed) #create a set for the non committed individuals
        for n, o in zip(noncommitted, opinions_to_assign):
            self.opinions[n]=set(o)
        
        #Expliciting the output if requested
        if verbose: print('Setup done.', self.N, 'nodes', "N_A:", N_A, "N_B:", N_B, "N_p:", N_p)

    def AgreeOnSimplex(self, simplex, said_word):
        """
        This function updates the simplex on the agreed word.
        """
        for n in simplex:
            try: #we use try because committed individuals have frozenset and they won't be updated 
                self.opinions[n].clear()
                self.opinions[n].add(said_word)
            except AttributeError: #it was a committed individual
                pass
    
    def ListenersLearnWord(self, listeners, said_word):
        """
        This function add said_word to the listeners' vocabulary.
        """
        for listener in listeners:
            try:
                self.opinions[listener].add(said_word)
            except AttributeError: #it was a committed individual
                pass
    
    def PlayOnSimplex(self, simplex):
        """
        This function simulate the interaction dynamics for a single group.
        """
        #Selecting speaker and listeners at random
        random.shuffle(simplex)
        speaker = simplex[0]
        listeners = simplex[1:]

        #Selecting a random word from the speaker's vocabulary to be said
        said_word = random.choice(list(self.opinions[speaker]))
        words_of_listeners = [self.opinions[listener] for listener in listeners]
        
        #Getting the words of listeners to be used for the agreement
        words_of_listeners_by_rule = set.intersection(*[set(w) for w in words_of_listeners])

        #Trying to agree acoording to the unanimity condition rule
        if (said_word in words_of_listeners_by_rule) and (random.random() <= self.beta):
            self.AgreeOnSimplex(simplex, said_word)
        else:
            self.ListenersLearnWord(listeners, said_word)

    def GetDensities(self):
        """
        This function returns the densities of each community.
        """
        single_opinion_counter = collections.Counter([list(opinions)[0] for opinions in self.opinions.values() if len(opinions)==1])
        n_Ap = single_opinion_counter["A"]/self.N
        n_B = single_opinion_counter["B"]/self.N
        n_AB = 1 - n_Ap - n_B
        return n_Ap, n_B, n_AB
    
    def run(self, path, t_max=100, check_every=10, print_every=1):
        """
        This function models the interaction of agents, updating their states over time. The simulation continues until all agents agree on a single word (consensus) or the maximum time, t_max, is reached.
        
        Args:
            path (str): The path to the directory where simulation density results will be saved.
            t_max (int):The maximum number of time steps (iterations) for the simulation. Defaults to 100.
            check_every (int): The number of iterations between saving simulation density results. Defaults to 10.
            print_every (int): The number of iterations between printing simulation updates. Defaults to 1.
        """
        self.t_max = t_max

        #Opening file to save densities results
        densities_path = path + 'simulation_densities_N%i_beta%.4f_p%.2f.csv'%(self.N, self.beta, self.p)
        f = open(densities_path, 'w')
        f.write('time,n_A+p,n_B,n_AB\n')

        #Running the simulation
        while self.t <= self.t_max:
            self.t += 1
            #Signaling the simulation is still running
            if self.t%print_every==0: print('t=%i'%self.t)
    
            #Selecting a random simplex to be played on
            simplex = random.choice(self.simplices)

            #Playing on simplex
            self.PlayOnSimplex(simplex)
                
            #Storing the values every check_every time steps:
            if self.t%check_every==0:
                n_Ap, n_B, n_AB = self.GetDensities()
                line = "%i,%.3f,%.3f,%.3f\n"%(self.t, n_Ap, n_B, n_AB)
                f.write(line)
                
                #Checking if we reached the absorbing state in order to interrupt the simulation:
                if n_Ap==1 or n_B==1:
                    f.close()   
                    print('Done! Reached the absorbing state.')
                    return None
                
        f.close()    
        print('Done! No more time left')
    
    def ReturnFilepath(self, path):
        results_filepath = path + 'simulation_densities_N%i_beta%.4f_p%.2f.csv'%(self.N, self.beta, self.p)
        return results_filepath

I ran simulations for the following values of $p$ and initial conditions:  
$$p = 0,   n_{A}=0.45$$  
$$p = 0.01,   n_{A}=0$$  
$$p = 0.03,   n_{A}=0$$  

In [3]:
dataset_dir = './Data/Sociopatterns/Processed_data/'
dataset = 'Thiers13'
n_minutes = 15
thr = 1

filename = dataset_dir+'aggr_'+str(n_minutes)+'min_cliques_thr'+str(thr)+'_'+dataset+'.json'
simplices = json.load(open(filename,'r'))

In [4]:
betas = np.linspace(0.,1,30)
p = 0.01
n_A = 0

t_max = 1e6
check_every = 500
print_every=50000

n_runs = 50

for run_id in range(n_runs):
    for beta in betas:
        print(run_id, beta)

        output_path = './Simulation_Results/Sociopatterns/%s_fixed_p%.2f_varbeta_run%i/'%(dataset, p, run_id)
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        simulation = Model(simplices)
        simulation.SetInitialConditions(beta=beta, p=p, n_A=n_A, verbose=True)
        simulation.run(output_path, t_max, check_every, print_every)

0 0.0
Setup done. 327 nodes N_A: 0 N_B: 324 N_p: 3
t=50000
t=100000
t=150000
t=200000
t=250000
t=300000
t=350000
t=400000
t=450000
t=500000
t=550000
t=600000
t=650000
t=700000
t=750000
t=800000
t=850000
t=900000
t=950000
t=1000000
Done! No more time left
0 0.034482758620689655
Setup done. 327 nodes N_A: 0 N_B: 324 N_p: 3
t=50000
t=100000
t=150000
t=200000
t=250000
t=300000
t=350000
t=400000
t=450000
t=500000
t=550000
t=600000
t=650000
t=700000
t=750000
t=800000
t=850000
t=900000
t=950000
t=1000000
Done! No more time left
0 0.06896551724137931
Setup done. 327 nodes N_A: 0 N_B: 324 N_p: 3
t=50000
t=100000
t=150000
t=200000
t=250000
t=300000
t=350000
t=400000
t=450000
t=500000
t=550000
t=600000
t=650000
t=700000
t=750000
t=800000
t=850000
t=900000
t=950000
t=1000000
Done! No more time left
0 0.10344827586206896
Setup done. 327 nodes N_A: 0 N_B: 324 N_p: 3
t=50000
t=100000
t=150000
t=200000
t=250000
t=300000
t=350000
t=400000
t=450000
t=500000
t=550000
t=600000
t=650000
t=700000
t=750000
t